- 교재 : ‘(파이토치 트랜스포머를 활용한) 자연어 처리와 컴퓨터비전 심층학습’
- 범위 : p591-p599 (YOLOv8)

In [1]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.9 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO


model = YOLO("../models/yolov8m-pose.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import cv2
import torch
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from google.colab import files

AttributeError: 'NoneType' object has no attribute 'clip'

In [10]:
import cv2
from google.colab.patches import cv2_imshow

capture = cv2.VideoCapture("/content/drive/MyDrive/2025-2/Euron/data/woman.mp4")

for _ in range(50):
    ret, frame = capture.read()
    if not ret:
        break
    cv2_imshow(frame)

capture.release()

In [11]:
import torch


def predict(frame, iou=0.7, conf=0.25):
    results = model(
        source=frame,
        device="0" if torch.cuda.is_available() else "cpu",
        iou=0.7,
        conf=0.25,
        verbose=False,
    )
    result = results[0]
    return result

In [12]:
def draw_boxes(result, frame):
    for boxes in result.boxes:
        x1, y1, x2, y2, score, classes = boxes.data.squeeze().cpu().numpy()
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
    return frame

In [14]:
import cv2
from google.colab.patches import cv2_imshow

capture = cv2.VideoCapture("/content/drive/MyDrive/2025-2/Euron/data/woman.mp4")

for _ in range(50):
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    ret, frame = capture.read()
    if not ret:
        break

    result = predict(frame)
    frame = draw_boxes(result, frame)
    cv2_imshow(frame)

capture.release()

In [16]:
from ultralytics.utils.plotting import Annotator
import cv2
from google.colab.patches import cv2_imshow

def draw_keypoints(result, frame):
    annotator = Annotator(frame, line_width=1)
    for kps in result.keypoints:
        kps = kps.data.squeeze()
        annotator.kpts(kps)
        nkps = kps.cpu().numpy()
        for idx, (x, y, score) in enumerate(nkps):
            if score > 0.5:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 0, 255), cv2.FILLED)
                cv2.putText(frame, str(idx), (int(x), int(y)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
    return frame

In [17]:
capture = cv2.VideoCapture("/content/drive/MyDrive/2025-2/Euron/data/woman.mp4")

for _ in range(50):
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    ret, frame = capture.read()
    if not ret:
        break

    result = predict(frame)
    frame = draw_boxes(result, frame)
    frame = draw_keypoints(result, frame)
    cv2_imshow(frame)

capture.release()